In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.calibration import LabelEncoder
from typing import Counter
from imblearn.over_sampling import SMOTE
import pickle

In [17]:
data=pd.read_excel(r"D:\Travancore Analytics\Version 3\Datasets\Augmented_dataset 3.xlsx")
data=data.dropna()

In [18]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]

In [ ]:
le=LabelEncoder()
Y_le=le.fit_transform(Y)
unique=le.classes_
unique_labels = unique.tolist()
print(unique)

In [ ]:
total_samples = len(X)*100
num_categories = len(Y.unique())
samples_per_category = total_samples // num_categories
print(samples_per_category)

In [ ]:
smote = SMOTE(sampling_strategy={label: samples_per_category for label in range(num_categories)}, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, Y_le)
print("Original class distribution:", Counter(Y_le))
print("Resampled class distribution:", Counter(Y_le))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
print(type(X_train))

In [ ]:
svm = SVC(kernel='rbf')  
svm.fit(X_train, y_train)

In [24]:
y_pred = svm.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'poly', 'sigmoid']
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(X_train, y_train)

In [ ]:
print("Best Parameters:", grid.best_params_)
print("Best Score:", grid.best_score_)

In [28]:
best_svm = grid.best_estimator_
y_pred_best = best_svm.predict(X_test)

In [ ]:
print("Tuned Model Accuracy:", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))

In [30]:
with open(r'D:\Travancore Analytics\Version 3\Models\support_vector_machines.pkl', 'wb') as f:
    pickle.dump(svm, f)    
    